# 04 — Hypothesis H1: Texture & Color Variability (GLCM)

**Objective**  
Quantify texture differences between healthy and mildew-infected cherry leaves using GLCM features to test H1:
> Mildew-infected leaves exhibit higher texture and color variability than healthy leaves.

**Inputs**  
- Image dataset: `inputs/cherry_leaves_dataset/{healthy, powdery_mildew}`

**Outputs**  
- CSV: `inputs/features/v1/glcm_features.csv` (per-image GLCM features)  
- CSV: `inputs/features/v1/glcm_stats.csv` (p-values & effect sizes)  
- CSV: `inputs/features/v1/glcm_feature_means.csv` (class-wise means/medians)  
- Plot: `plots/v2/glcm_boxplots.png` (boxplots per feature by class)

**Notes**  
Images are resized to 100×100, converted to grayscale, quantized to fixed gray levels, and GLCM features are averaged across multiple distances and angles.

In [2]:
from pathlib import Path
import sys

def find_project_root(start: Path) -> Path:
    """Walk up until a folder containing 'src' is found, else return start."""
    p = start
    for _ in range(5):
        if (p / "src").exists():
            return p
        p = p.parent
    return start

PROJECT_ROOT = find_project_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from src.paths import DATA_DIR, PLOTS_DIR, PROJECT_ROOT

print("PROJECT_ROOT:", PROJECT_ROOT)
print("DATA_DIR:", DATA_DIR)
print("PLOTS_DIR:", PLOTS_DIR)

PROJECT_ROOT: C:\Users\ksstr\Documents\Coding\milestone-project-5
DATA_DIR: C:\Users\ksstr\Documents\Coding\milestone-project-5\inputs\cherry_leaves_dataset
PLOTS_DIR: C:\Users\ksstr\Documents\Coding\milestone-project-5\plots\v1


In [3]:
# Configuration and output directories
from pathlib import Path

CLASSES = ("healthy", "powdery_mildew")
ALLOWED = {".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"}

IMG_SIZE = (100, 100)           # (width, height)
GRAY_LEVELS = 32                # quantization levels for GLCM
DISTANCES = [1, 2, 3]           # pixel distances
ANGLES = [0, 0.25, 0.5, 0.75]   # angles in units of pi (0, π/4, π/2, 3π/4)
FEATURES = ["contrast", "energy", "homogeneity", "correlation"]

# Output locations
FEATURES_DIR = Path("inputs") / "features" / "v1"
FEATURES_DIR.mkdir(parents=True, exist_ok=True)

PLOTS_V2_DIR = PLOTS_DIR.parent / "v2"
PLOTS_V2_DIR.mkdir(parents=True, exist_ok=True)

print("FEATURES_DIR:", FEATURES_DIR.resolve())
print("PLOTS_V2_DIR:", PLOTS_V2_DIR.resolve())

FEATURES_DIR: C:\Users\ksstr\Documents\Coding\milestone-project-5\jupyter_notebooks\inputs\features\v1
PLOTS_V2_DIR: C:\Users\ksstr\Documents\Coding\milestone-project-5\plots\v2


In [4]:
# Image loading, grayscale quantization, and GLCM feature computation
import numpy as np
from PIL import Image
from skimage.feature import graycomatrix, graycoprops
from pathlib import Path

def iter_image_paths(base_dir: Path, label: str):
    """Yield file paths of valid images inside a class folder."""
    folder = base_dir / label
    if not folder.exists():
        return
    for p in folder.iterdir():
        if p.is_file() and p.suffix in ALLOWED:
            yield p

def load_gray_quantized(path: Path, size=IMG_SIZE, levels=GRAY_LEVELS) -> np.ndarray:
    """
    Read image, resize to `size`, convert to 8-bit grayscale, then
    uniformly quantize to integer gray levels in [0, levels-1].
    """
    with Image.open(path) as im:
        im = im.convert("L").resize(size)
        arr = np.asarray(im, dtype=np.uint8)           # [0..255]
    q = (arr.astype(np.float32) / 255.0) * (levels - 1)
    return q.astype(np.uint8)                          # [0..levels-1]

def compute_glcm_features(qimg: np.ndarray,
                          distances=DISTANCES,
                          angles=ANGLES,
                          levels=GRAY_LEVELS,
                          features=FEATURES) -> dict:
    """
    Compute GLCM features averaged over all distance/angle combinations.
    Angles are specified in units of pi (e.g., 0.25 = pi/4).
    """
    angles_rad = np.array(angles, dtype=np.float64) * np.pi
    glcm = graycomatrix(
        qimg,
        distances=distances,
        angles=angles_rad,
        levels=levels,
        symmetric=True,
        normed=True
    )

    out = {}
    for feat in features:
        vals = graycoprops(glcm, feat)  # shape: (len(distances), len(angles))
        out[feat] = float(vals.mean())
    return out

### Functional Validation — GLCM Feature Pipeline

A quick sanity check to verify that the GLCM feature extraction functions
(`iter_image_paths`, `load_gray_quantized`, `compute_glcm_features`)
return plausible outputs before running the full dataset loop.

In [ ]:
from pprint import pprint

# Pick one sample image from each class (if available)
for cls in CLASSES:
    paths = list(iter_image_paths(DATA_DIR, cls))
    if not paths:
        print(f"No images found in class: {cls}")
        continue

    # Load and quantize one example image
    test_img = load_gray_quantized(paths[0])
    print(f"\n[{cls}] Test image shape:", test_img.shape,
          "| Range:", int(test_img.min()), "-", int(test_img.max()))

    # Compute texture features
    feats = compute_glcm_features(test_img)
    print(f"GLCM features for {cls}:")
    pprint(feats)